<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
%pip install numpy -q


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import functools  # For lru_cache memoization
import numpy as np
import heapq  # for priority queue
import copy  # for deepcopy of lists
from queue import PriorityQueue

In [ ]:
GOAL_STATE_8 = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
]  # goal state for 8-puzzle

In [ ]:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle

    Attributes:
        - state (list of lists): The current state of the puzzle.
        - parent (PuzzleNode): The parent node in the search tree.
        - g (int): The cost from the start node to this node.
        - h (int): The heuristic estimate of the cost from this node to the goal.
        - f (int): The total estimated cost (f = g + h).
    """

    def __init__(self, state, parent=None, g=0, heuristic=None):
        """
        Initialize a PuzzleNode.

        Input:
            -state: The current state of the puzzle (list of lists).
            -parent: The parent node (PuzzleNode).
            -g: The cost from the start node to this node.
            -heuristic: A function that calculates the heuristic value.
        """
        self.state = state
        self.parent = parent  # Store parent node
        self.g = g  # Cost from the start node to this node (g value)
        self.heuristic = heuristic
        self.h = (
            self.heuristic(self.state) if heuristic else 0
        )  # Heuristic value (h value)
        self.f = self.g + self.h  # Total cost (f = g + h)

    def __str__(self):
        """
        Print the state of the node

        returns:
            -str: node's state
        """
        grid = ""
        n = len(self.state)
        for row in self.state:
            grid += (
                "| "
                + " | ".join(f"{cell:2}" if cell != 0 else "  " for cell in row)
                + " |\n"
            )
        return grid

    def __lt__(self, other):
        """
        Define less-than comparison between two PuzzleNode instances based on their f value.

        returns:
            -bool: True if the current node has a lower f value than the other node, False otherwise.
        """
        return self.f < other.f

    def expand(self):
        """
        Expand the current node.

        returns:
            -children: A list of the child nodes.
        """
        children = []

        # Store the position of the empty tile
        empty_tile = None
        for i in range(len(self.state)):
            for j in range(len(self.state[i])):
                if self.state[i][j] == 0:
                    empty_tile = (i, j)
                    break
            if empty_tile:
                break

        # Generate child nodes by moving nearby tiles to the empty tile
        for i, j in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            child_i, child_j = empty_tile[0] + i, empty_tile[1] + j
            # Check if the child node is within the grid
            if 0 <= child_i < len(self.state) and 0 <= child_j < len(self.state[0]):
                # Create a child state by copying the current state
                child_state = [row[:] for row in self.state]
                # Swap the empty tile and the nearby tile
                (
                    child_state[empty_tile[0]][empty_tile[1]],
                    child_state[child_i][child_j],
                ) = (child_state[child_i][child_j], 0)
                # Append the child node to the list and increment the cost
                children.append(
                    PuzzleNode(child_state, self, self.g + 1, self.heuristic)
                )

        return children

In [5]:
# TEST - Create an instance of PuzzleNode
initial_state = [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
initial_node = PuzzleNode(initial_state, cost=1, h=2)
print("Initial Puzzle Node:")
print(initial_node)

Initial Puzzle Node:
1 2 3 
4 5 0 
6 7 8 



<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [6]:
import functools  # For lru_cache memoization
import numpy as np


# Robust Memoization decorator that handles list inputs
def memoize(func):
    """
    Memoization decorator that handles list inputs by converting them to tuples.
    Caches the results of the function for previously seen inputs to avoid redundant computations.

    This decorator is necessary because list arguments are not hashable in Python,
    which is required for caching with functools.lru_cache or dictionaries.
    It converts list arguments (including nested lists representing puzzle states)
    into tuples before using them as keys in the cache.
    """
    cache = {}  # Initialize the cache for memoized results

    def memoized_func(*args, **kwargs):
        """Memoized version of the function."""
        # Convert list arguments to tuples for hashability
        hashable_args = []
        for arg in args:
            if isinstance(arg, list):
                # Convert nested lists (puzzle states) to nested tuples
                if all(
                    isinstance(item, list) for item in arg if isinstance(item, list)
                ):
                    hashable_arg = tuple(
                        tuple(row) for row in arg
                    )  # Convert 2D list to tuple of tuples
                else:
                    hashable_arg = tuple(arg)  # Convert 1D list to tuple
            else:
                hashable_arg = arg  # Argument is already hashable
            hashable_args.append(hashable_arg)

        # Create a hashable key from the combined arguments (args and kwargs)
        key = (tuple(hashable_args), frozenset(kwargs.items()) if kwargs else None)

        if key not in cache:  # Check if the result for this key is already cached
            cache[key] = func(
                *args, **kwargs
            )  # Compute and cache the result if not in cache
        return cache[key]  # Return the cached result

    return memoized_func


# Function to create the goal state for an n x n puzzle
def create_goal_state(n):
    """
    Creates the goal state for an n x n puzzle.
    The goal state is defined as tiles numbered 0 to n^2 - 1 in row-major order,
    with 0 representing the blank tile in the top-left corner.

    Args:
        n (int): The dimension of the puzzle (n x n).

    Returns:
        list of lists: The goal state as a 2D list.

    For example, for n=3, the goal state is:
    [[0, 1, 2],
     [3, 4, 5],
     [6, 7, 8]]
    """
    goal_state = [
        [0 for _ in range(n)] for _ in range(n)
    ]  # Initialize an n x n grid with 0s
    tile_val = 0  # Start tile value from 0 for the goal state
    for i in range(n):
        for j in range(n):
            goal_state[i][
                j
            ] = tile_val  # Assign increasing tile values in row-major order
            tile_val += 1
    return goal_state


# Function to get the goal positions of tiles in the goal state
def goal_state_position(n):
    """
    Pre-calculates and stores the goal positions (row, column) for each tile value in the goal state.
    This allows for quick lookups of goal positions when calculating heuristics like Manhattan distance.

    Args:
        n (int): The dimension of the puzzle (n x n).

    Returns:
        dict: A dictionary where keys are tile values (0 to n^2 - 1) and values are their goal positions as tuples (row, column).

    For example, for n=3, the position dictionary would be like:
    {0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (1, 0), 4: (1, 1), 5: (1, 2), 6: (2, 0), 7: (2, 1), 8: (2, 2)}
    """
    goal_state = create_goal_state(
        n
    )  # Generate the goal state for the given puzzle size
    position = {}  # Initialize a dictionary to store goal positions
    for i in range(n):
        for j in range(n):
            position[goal_state[i][j]] = (
                i,
                j,
            )  # Store the row and column as the goal position for each tile value
    return position


# Heuristic function 1: Misplaced Tiles heuristic
@memoize  # Apply memoization decorator to cache results of h1
def h1(state):
    """
    Calculates the Misplaced Tiles heuristic for a given puzzle state.
    This heuristic counts the number of tiles that are not in their goal positions in the given state.
    It is an admissible heuristic as each misplaced tile requires at least one move to reach its correct position.

    Args:
        state (list of lists): The current state of the puzzle as a 2D list.

    Returns:
        int: The number of misplaced tiles in the given state.
    """
    goal_state = create_goal_state(len(state))  # Get the goal state for comparison
    misplaced_tiles_count = 0  # Initialize counter for misplaced tiles
    n = len(state)  # Get the puzzle dimension (n x n)

    for i in range(n):
        for j in range(n):
            # Check if the tile is not the blank tile (0) and is not in its goal position
            if state[i][j] != goal_state[i][j] and state[i][j] != 0:
                misplaced_tiles_count += (
                    1  # Increment the count for each misplaced tile
                )
    return misplaced_tiles_count  # Return the total count of misplaced tiles


# Heuristic function 2: Manhattan Distance heuristic
@memoize  # Apply memoization decorator to cache results of h2
def h2(state):
    """
    Calculates the Manhattan Distance heuristic for a given puzzle state.
    This heuristic computes the sum of the Manhattan distances of each tile from its current position
    to its goal position. Manhattan distance is the sum of the absolute differences of their row and column indices.
    It is an admissible heuristic because each tile must move at least its Manhattan distance to reach its goal position.

    Args:
        state (list of lists): The current state of the puzzle as a 2D list.

    Returns:
        int: The total Manhattan distance for the given state.
    """
    manhattan_distance_total = 0  # Initialize total Manhattan distance
    goal_positions = goal_state_position(
        len(state)
    )  # Get pre-calculated goal positions of tiles for the puzzle size
    n = len(state)  # Get the puzzle dimension (n x n)

    for i in range(n):
        for j in range(n):
            tile_value = state[i][
                j
            ]  # Get the value of the tile at the current position
            if (
                tile_value != 0
            ):  # Exclude the blank tile (0) from the heuristic calculation
                goal_row, goal_col = goal_positions[
                    tile_value
                ]  # Get the goal row and column for the tile value
                # Calculate Manhattan distance for the tile and add to the total distance
                manhattan_distance_total += abs(i - goal_row) + abs(j - goal_col)
    return manhattan_distance_total  # Return the total Manhattan distance


# Linear Conflict heuristic (h3) - Dominates Manhattan Distance (for extension)
@memoize
def h3(state):
    """
    Calculates the Linear Conflict heuristic, which dominates Manhattan Distance.
    ... (docstring remains the same) ...
    """
    state_tuple = tuple(
        tuple(row) for row in state
    )  # Convert input state to tuple of tuples for memoization
    h_man = h2(state)
    linear_conflicts = 0
    n = len(state)
    goal_state = create_goal_state(n)

    # Check for row conflicts
    for i in range(n):
        tiles_in_row = []
        for j in range(n):
            tile = state[i][j]
            if tile != 0 and goal_state[i].count(tile):
                tiles_in_row.append((tile, j))
        tiles_in_row.sort(key=lambda x: goal_state[i].index(x[0]))
        for k in range(len(tiles_in_row)):
            for l in range(k + 1, len(tiles_in_row)):
                if tiles_in_row[k][1] > tiles_in_row[l][1]:
                    linear_conflicts += 2

    # Check for column conflicts (similar logic as row conflicts)
    for j in range(n):
        tiles_in_col = []
        for i in range(n):
            tile = state[i][j]
            goal_col = [goal_state[row][j] for row in range(n)]
            if tile != 0 and goal_col.count(tile):
                tiles_in_col.append((tile, i))
        tiles_in_col.sort(
            key=lambda x: [goal_state[row][j] for row in range(n)].index(x[0])
        )
        for k in range(len(tiles_in_col)):
            for l in range(k + 1, len(tiles_in_col)):
                if tiles_in_col[k][1] > tiles_in_col[l][1]:
                    linear_conflicts += 2

    return h_man + linear_conflicts


# Heuristic list - contains all implemented heuristic functions
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [7]:
import heapq  # for priority queue
import copy  # for deepcopy of lists

# Import any packages or define any helper functions you need here
# GOAL_STATE_8 = [[0, 1, 2], [3, 4, 5,], [6, 7, 8]] # Removed hardcoded goal state
# GOAL_STATE_8_TUPLE = tuple(tuple(row) for row in GOAL_STATE_8) # No longer needed - dynamic goal state


def get_blank_pos(state):
    """Find blank tile (0) position (row, col)."""
    n = len(state)
    for i in range(n):
        for j in range(n):
            if state[i][j] == 0:
                return i, j
    return None  # Should not happen in valid puzzles


def get_successors(node, heuristic):
    """Generate successor PuzzleNodes for a given node."""
    current_state = node.state
    blank_row, blank_col = get_blank_pos(current_state)
    n = len(current_state)
    successors = []
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up

    for dr, dc in moves:
        new_row, new_col = blank_row + dr, blank_col + dc
        if 0 <= new_row < n and 0 <= new_col < n:  # Valid move within grid
            new_state = [list(row) for row in current_state]  # Create new state
            # Swap blank tile
            new_state[blank_row][blank_col], new_state[new_row][new_col] = (
                new_state[new_row][new_col],
                new_state[blank_row][blank_col],
            )
            h_value = heuristic(new_state)  # Calculate heuristic for successor
            successor_node = PuzzleNode(
                state=new_state, parent=node, cost=node.cost + 1, h=h_value
            )  # Create successor node
            successors.append(successor_node)
    return successors


def is_valid_state(state):
    """Check if input state is a valid n-puzzle."""
    if not isinstance(state, list):
        return False
    n = len(state)
    if n < 2:
        return False  # Minimum 2x2 puzzle
    for row in state:
        if not isinstance(row, list) or len(row) != n:
            return False

    expected_numbers = set(range(n * n))
    actual_numbers = set()
    for i in range(n):
        for j in range(n):
            if not isinstance(state[i][j], int):
                return False
            actual_numbers.add(state[i][j])

    return actual_numbers == expected_numbers  # Check for correct numbers


def count_inversions(state):
    """Count inversions in a puzzle state (excluding blank tile)."""
    n = len(state)
    inversions = 0
    linear_state = []

    for i in range(n):
        for j in range(n):
            if state[i][j] != 0:
                linear_state.append(state[i][j])

    for i in range(len(linear_state)):
        for j in range(i + 1, len(linear_state)):
            if linear_state[i] > linear_state[j]:
                inversions += 1
    return inversions


def is_solvable(state):
    """Determine if n-puzzle state is solvable using inversion count."""
    n = len(state)
    inversions = count_inversions(state)

    if n % 2 == 1:  # Odd-sized puzzle
        return inversions % 2 == 0
    else:  # Even-sized puzzle
        blank_row_from_bottom = n - get_blank_pos(state)[0]
        return (blank_row_from_bottom % 2 == 0) == (
            inversions % 2 == 1
        )  # XOR for solvability condition


# Main solvePuzzle function.
def solvePuzzle(state, heuristic):
    """
    Solves the n**2-1 puzzle using A* search algorithm.

    Args:
        state (list of lists): Initial puzzle state.
        heuristic (function): Heuristic function (h1, h2, or h3).

    Returns:
        tuple: (steps, expansions, max_frontier, opt_path, err).
               - steps (int): Number of steps in optimal path.
               - expansions (int): Number of nodes expanded.
               - max_frontier (int): Maximum frontier size.
               - opt_path (list): Optimal path (list of states).
               - err (int): Error code (0: success, -1: invalid state, -2: unsolvable).
    """
    if not is_valid_state(state):
        return 0, 0, 0, None, -1  # Return error -1 for invalid state

    if not is_solvable(state):
        return 0, 0, 0, None, -2  # Return error -2 for unsolvable state

    n = len(state)  # Puzzle dimension
    goal_state = create_goal_state(n)  # Dynamically create goal state
    goal_state_tuple = tuple(
        tuple(row) for row in goal_state
    )  # Tuple for efficient comparison

    print(f"Puzzle dimension (n): {n}")  # ADD THIS LINE
    print("Dynamically generated Goal State:")  # ADD THIS LINE
    for row in goal_state:  # ADD THIS LINE
        print(row)  # ADD THIS LINE
    print(f"Goal State Tuple (for comparison): {goal_state_tuple}")  # ADD THIS LINE

    initial_h_value = heuristic(state)
    initial_node = PuzzleNode(state, parent=None, cost=0, h=initial_h_value)

    frontier = [
        (initial_node.f_value, initial_node)
    ]  # Priority queue (min-heap) by f_value
    heapq.heapify(frontier)  # Initialize heap
    explored_set = set()  # Set for explored states (tuple of tuples for hashability)
    max_frontier_size = 1
    nodes_expanded = 0

    while frontier:  # A* search loop
        f_val, current_node = heapq.heappop(frontier)  # Get node with lowest f_value

        current_state_tuple = tuple(
            tuple(row) for row in current_node.state
        )  # For explored set check

        if current_state_tuple == goal_state_tuple:  # Goal state reached
            path = []
            node = current_node
            while node:  # Backtrack to build path
                path.append(node.state)
                node = node.parent
            return (
                current_node.cost,
                nodes_expanded,
                max_frontier_size,
                path[::-1],
                0,
            )  # Return success

        if current_state_tuple in explored_set:  # Already explored state
            continue  # Skip and continue

        explored_set.add(current_state_tuple)  # Mark state as explored
        nodes_expanded += 1  # Increment expanded node count

        successors = get_successors(
            current_node, heuristic
        )  # Generate valid next states
        for successor in successors:
            heapq.heappush(
                frontier, (successor.f_value, successor)
            )  # Add successors to frontier
        max_frontier_size = max(
            max_frontier_size, len(frontier)
        )  # Update max frontier size

    return (
        0,
        0,
        max_frontier_size,
        None,
        0,
    )  # Should not reach here for solvable puzzles

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [8]:
## Test for state not correctly defined

incorrect_state = [[0, 1, 2], [2, 3, 4], [5, 6, 7]]
_, _, _, _, err = solvePuzzle(incorrect_state, lambda state: 0)
assert err == -1

In [9]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = (
    [[2, 3, 7], [1, 8, 0], [6, 5, 4]],
    [[7, 0, 8], [4, 6, 1], [5, 3, 2]],
    [[5, 7, 6], [2, 4, 3], [8, 1, 0]],
)

# Test the values returned by the heuristic functions
h_mt_vals = [7, 8, 7]
h_man_vals = [15, 17, 18]

for i in range(0, 3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert h_mt == h_mt_vals[i]
    assert h_man == h_man_vals[i]

In [10]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [
    [[2, 3, 7], [1, 8, 0], [6, 5, 4]],
    [[2, 3, 7], [1, 8, 4], [6, 5, 0]],
    [[2, 3, 7], [1, 8, 4], [6, 0, 5]],
    [[2, 3, 7], [1, 0, 4], [6, 8, 5]],
    [[2, 0, 7], [1, 3, 4], [6, 8, 5]],
    [[0, 2, 7], [1, 3, 4], [6, 8, 5]],
    [[1, 2, 7], [0, 3, 4], [6, 8, 5]],
    [[1, 2, 7], [3, 0, 4], [6, 8, 5]],
    [[1, 2, 7], [3, 4, 0], [6, 8, 5]],
    [[1, 2, 0], [3, 4, 7], [6, 8, 5]],
    [[1, 0, 2], [3, 4, 7], [6, 8, 5]],
    [[1, 4, 2], [3, 0, 7], [6, 8, 5]],
    [[1, 4, 2], [3, 7, 0], [6, 8, 5]],
    [[1, 4, 2], [3, 7, 5], [6, 8, 0]],
    [[1, 4, 2], [3, 7, 5], [6, 0, 8]],
    [[1, 4, 2], [3, 0, 5], [6, 7, 8]],
    [[1, 0, 2], [3, 4, 5], [6, 7, 8]],
    [[0, 1, 2], [3, 4, 5], [6, 7, 8]],
]

astar_steps = [17, 25, 28]
for i in range(0, 3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[0]
    )
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1]
    )
    # Test whether the number of optimal steps is correct and the same
    assert steps_mt == steps_man == astar_steps[i]
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert expansions_man < expansions_mt
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert opt_path_mt == opt_path_soln

Puzzle dimension (n): 3
Dynamically generated Goal State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
Goal State Tuple (for comparison): ((0, 1, 2), (3, 4, 5), (6, 7, 8))
Puzzle dimension (n): 3
Dynamically generated Goal State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
Goal State Tuple (for comparison): ((0, 1, 2), (3, 4, 5), (6, 7, 8))
Puzzle dimension (n): 3
Dynamically generated Goal State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
Goal State Tuple (for comparison): ((0, 1, 2), (3, 4, 5), (6, 7, 8))
Puzzle dimension (n): 3
Dynamically generated Goal State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
Goal State Tuple (for comparison): ((0, 1, 2), (3, 4, 5), (6, 7, 8))
Puzzle dimension (n): 3
Dynamically generated Goal State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
Goal State Tuple (for comparison): ((0, 1, 2), (3, 4, 5), (6, 7, 8))
Puzzle dimension (n): 3
Dynamically generated Goal State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
Goal State Tuple (for comparison): ((0, 1, 2), (3, 4, 5), (6, 7, 8))


In [13]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [
    [1, 2, 6, 3],
    [0, 9, 5, 7],
    [4, 13, 10, 11],
    [8, 12, 14, 15],
]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[0]
)
steps_man, expansions_man, _, _, _ = solvePuzzle(
    working_initial_state_15_puzzle, heuristics[1]
)

# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert expansions_mt >= expansions_man

# Test whether the number of optimal steps is correct and the same
# assert steps_mt == steps_man == 9

print(steps_man,)

0 0


<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [ ]:
## Puzzle solvability test

unsolvable_initial_state = [[7, 5, 6], [2, 4, 3], [8, 1, 0]]
_, _, _, _, err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert err == -2

In [ ]:
## Extra heuristic function test.
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0, 3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[2]
    )
    steps_man, expansions_man, _, _, _ = solvePuzzle(
        working_initial_states_8_puzzle[i], heuristics[1]
    )
    # Test whether the number of optimal steps is correct and the same
    assert steps_new == steps_man == astar_steps[i]
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert dom > 0

In [ ]:
## Memoization test - will be carried out after submission